# 金融风控学习赛
## 1.读取数据并进行观察（略）

In [7]:
import pandas as pd
import numpy as np 
from numpy import inf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#读取基础数据阶段
Train_data = pd.read_csv('train.csv',encoding='utf-8',delimiter=',')
Test_data =pd.read_csv('testA.csv',encoding='utf-8',delimiter=',' )
n_train = len(Train_data)
print('Train data shape:',Train_data.shape)
print('TestA data shape:',Test_data.shape)

import os
path = os.path.abspath(os.path.dirname(os.getcwd()) + os.path.sep + ".")
datapath = path + '/2.Risk_of_Loan/Processdata/'

Train data shape: (800000, 47)
TestA data shape: (200000, 46)


In [40]:
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.preprocessing import StandardScaler,MinMaxScaler

from sklearn.metrics import accuracy_score,mean_squared_error,roc_auc_score

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [9]:
# 用来减少内存使用的方程
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum()  / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum()  / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
All_Y = Train_data['isDefault']
All_Y.to_csv(datapath+'Y_Final.csv',index=False)

All_data = pd.concat([Train_data,Test_data]).reset_index(drop=True)
print('All Features in raw data are:',All_data.columns.shape,'\n',All_data.columns)
Useless_Feature='isDefault/policyCode' 
Useless_F=Useless_Feature.split('/')
print('Useless Features are:',Useless_F)
All_data = All_data.drop(Useless_F,axis=1)
print('Useless Features are already dropped...',All_data.columns.shape)

All Features in raw data are: (47,) 
 Index(['id', 'loanAmnt', 'term', 'interestRate', 'installment', 'grade',
       'subGrade', 'employmentTitle', 'employmentLength', 'homeOwnership',
       'annualIncome', 'verificationStatus', 'issueDate', 'isDefault',
       'purpose', 'postCode', 'regionCode', 'dti', 'delinquency_2years',
       'ficoRangeLow', 'ficoRangeHigh', 'openAcc', 'pubRec',
       'pubRecBankruptcies', 'revolBal', 'revolUtil', 'totalAcc',
       'initialListStatus', 'applicationType', 'earliesCreditLine', 'title',
       'policyCode', 'n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8',
       'n9', 'n10', 'n11', 'n12', 'n13', 'n14'],
      dtype='object')
Useless Features are: ['isDefault', 'policyCode']
Useless Features are already dropped... (45,)


In [92]:
#用于快速计算LGB未调参状态下的AUC值以便比较
def fast_test_lgbauc(All_data): 
    All_data =All_data.drop(['id','isDefault'],axis=1)
    All_train =  All_data[:800000] 
    All_test = All_data[800000:] 
    print('All_train size',All_train.shape,'All_test size',All_test.shape)
    del All_data

    X_Train, X_Test, y_Train, y_Test = train_test_split(All_train, All_Y, test_size=0.2, random_state=233)

    LGB = LGBMClassifier(random_state=9,n_jobs = -1).fit(X_Train,y_Train)
    print("LGB AUC Train score: %.4f  " % roc_auc_score(y_Train,LGB.predict_proba(X_Train)[:,1]),end='')
    print("LGB AUC Test score: %.4f" % roc_auc_score(y_Test,LGB.predict_proba(X_Test)[:,1]))
    importances = LGB.feature_importances_
    Featurename = LGB.feature_name_
    feature_importance = pd.DataFrame({'feature_name':Featurename,'importance':importances})
    return feature_importance

#该函数用于计算缺失率
def calculate_missratio(Dataset,n=5):
    All_data = Dataset
    total = All_data.isnull().sum().sort_values(ascending = False)
    percent = (All_data.isnull().sum()/All_data.isnull().count()).sort_values(ascending = False)
    missing_data = pd.concat([total,percent],axis=1,keys=['total','Percent'])
    print(missing_data.head(n))

## 2 采用随机森林填充相关缺失值

### 2.1 对非int/float特征进行一些处理方便进行填充
主要填充的参数如下
~~~
grade_replace = {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7}
subGrade_replace = {'A1':1,'A2':2,'A3':3,'A4':4,'A5':5,'B1':6,'B2':7,'B3':8,'B4':9,'B5':10,
                    'C1':11,'C2':12,'C3':13,'C4':14,'C5':15,'D1':16,'D2':17,'D3':18,'D4':19,'D5':20,
                    'E1':21,'E2':22,'E3':23,'E4':24,'E5':25,'F1':26,'F2':27,'F3':28,'F4':29,'F5':30,
                    'G1':31,'G2':32,'G3':33,'G4':34,'G5':35}
year_replace = {'< 1 year' :0, '1 year':1, '2 years':2, '3 years':3, '4 years':4, '5 years':5, 
                 '6 years':6, '7 years':7,  '8 years':8, '9 years':9, '10+ years' :10}
                 
                 

In [11]:
print('Classdata Processing......:')
print('Example of data\n',All_data[['grade','subGrade','employmentLength']].head(2))

grade_replace = {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7}
subGrade_replace = {'A1':1,'A2':2,'A3':3,'A4':4,'A5':5,'B1':6,'B2':7,'B3':8,'B4':9,'B5':10,
                    'C1':11,'C2':12,'C3':13,'C4':14,'C5':15,'D1':16,'D2':17,'D3':18,'D4':19,'D5':20,
                    'E1':21,'E2':22,'E3':23,'E4':24,'E5':25,'F1':26,'F2':27,'F3':28,'F4':29,'F5':30,
                    'G1':31,'G2':32,'G3':33,'G4':34,'G5':35}
year_replace = {'< 1 year' :0, '1 year':1, '2 years':2, '3 years':3, '4 years':4, '5 years':5, 
                 '6 years':6, '7 years':7,  '8 years':8, '9 years':9, '10+ years' :10}

All_data['grade'] = All_data['grade'].replace(grade_replace)     
All_data['subGrade'] = All_data['subGrade'].replace(subGrade_replace)     
All_data['employmentLength'] = All_data['employmentLength'].replace(year_replace)     
print('Classdata Processing finished')
print('Example of data\n',All_data[['grade','subGrade','employmentLength']].head(2))

Classdata Processing......:
Example of data
   grade subGrade employmentLength
0     E       E2          2 years
1     D       D2          5 years
Classdata Processing finished
Example of data
    grade  subGrade  employmentLength
0      5        22               2.0
1      4        17               5.0


### 2.2 对日期特征进行处理

In [13]:
print('DataFormat Processing......:')
All_data['issueDate_refmt']=pd.to_datetime(All_data['issueDate'],format ='%Y-%m-%d')
All_data['earliesCreditLine_refmt']=pd.to_datetime(All_data['earliesCreditLine'],format ='%b-%Y')
print('DataFormat Features are already changed...',All_data.columns.shape)

All_data['issueDate_month']=All_data['issueDate_refmt'].dt.month
All_data['issueDate_year']=All_data['issueDate_refmt'].dt.year
All_data['earliesCreditLine_month']=All_data['earliesCreditLine_refmt'].dt.month
All_data['earliesCreditLine_year']=All_data['earliesCreditLine_refmt'].dt.month
All_data['Credit_range_months']=round((All_data['issueDate_refmt']-All_data['earliesCreditLine_refmt']).dt.days/30,1)

print('DataFormat Features are already added...',All_data.columns.shape)

droplisttext='issueDate/earliesCreditLine/issueDate_refmt/earliesCreditLine_refmt'
droplist = droplisttext.split('/')
All_data = All_data.drop(droplist,axis=1)
print('DataFormat Features temp are already removed...',All_data.columns.shape)

DataFormat Processing......:
DataFormat Features are already changed... (47,)
DataFormat Features are already added... (52,)
DataFormat Features temp are already removed... (48,)


In [18]:
All_data.to_csv(datapath+'All_data_NewStep1.csv',index=False)

无任何处理情况下基础LGB的AUC值如下
```
All_train size (800000, 48) All_test size (200000, 48)
LGB AUC Train score: 0.7398  LGB AUC Test score: 0.7296
```

### 2.3 开始将n0-n14的匿名变量用进行稍微分桶以便提高Random填充精度

In [76]:
All_data = pd.read_csv(datapath + 'All_data_NewStep1.csv',encoding='utf-8',delimiter=',')
All_Y = pd.read_csv(datapath + 'Y_Final.csv',encoding='utf-8',delimiter=',')
All_data['isDefault'] = All_Y
Normal_Columns=set(All_data.columns)

In [77]:
All_data['n1'] = np.floor_divide(All_data['n1'], 3)
All_data['n2'] = np.floor_divide(All_data['n2'], 5)
All_data['n3'] = np.floor_divide(All_data['n3'], 4)
All_data['n4'] = np.floor_divide(All_data['n4'], 5)
All_data['n5'] = np.floor_divide(All_data['n5'], 6)
All_data['n6'] = np.floor_divide(All_data['n6'], 10)
All_data['n7'] = np.floor_divide(All_data['n7'], 10)
All_data['n8'] = np.floor_divide(All_data['n8'], 10)
All_data['n9'] = np.floor_divide(All_data['n9'], 2)
All_data['n10'] = np.floor_divide(All_data['n10'], 5)

In [78]:
All_data['n1'].loc[All_data['n1']>7] = 7
All_data['n2'].loc[All_data['n2']>7] = 7
All_data['n3'].loc[All_data['n3']>8] = 8
All_data['n4'].loc[All_data['n4']>7] = 7
All_data['n5'].loc[All_data['n5']>9] = 9
All_data['n6'].loc[All_data['n6']>9] = 9
All_data['n7'].loc[All_data['n7']>5] = 5
All_data['n8'].loc[All_data['n8']>9] = 9
All_data['n9'].loc[All_data['n9']>12] = 12
All_data['n10'].loc[All_data['n10']>11] = 11
All_data['n11'] = All_data['n11'].replace({3:2,4:2})
All_data['n12'] = All_data['n12'].replace({3:2,4:2})
All_data['n13'].loc[All_data['n13']>10] = 10
All_data['n14'].loc[All_data['n14']>10] = 10

### 2.4 填充缺失值少于100的特征

In [79]:
print('Fill Data with only 1 NaN......:')
FeatureFillTar = 'employmentTitle/title/postCode'
FeatureFill=FeatureFillTar.split('/')
print('Feature Filling Target:\n',FeatureFill)
for var in FeatureFill:
    fillvalue = float(All_data[var].mode())
    All_data[var] = All_data[var].fillna(fillvalue)
    print('Feature:',var,'is already filled by mode')

Fill Data with only 1 NaN......:
Feature Filling Target:
 ['employmentTitle', 'title', 'postCode']
Feature: employmentTitle is already filled by mode
Feature: title is already filled by mode
Feature: postCode is already filled by mode


In [80]:
All_data.to_csv(datapath+'All_data_NewStep2.csv',index=False)

### 2.5用随机森林填充缺失值(离散值)

In [11]:
All_data = pd.read_csv(datapath + 'All_data_NewStep2.csv',encoding='utf-8',delimiter=',')

Full_feature =  All_data.describe().T['count'][All_data.describe().T['count'].values == All_data.shape[0]].index
print('The features without null data:',Full_feature.shape,'\n',Full_feature)
Miss_feature =  All_data.describe().T['count'][All_data.describe().T['count'].values != All_data.shape[0]].index
print('The features have null data:',Miss_feature.shape,'\n',Miss_feature)

The features without null data: (29,) 
 Index(['id', 'loanAmnt', 'term', 'interestRate', 'installment', 'grade',
       'subGrade', 'employmentTitle', 'homeOwnership', 'annualIncome',
       'verificationStatus', 'purpose', 'postCode', 'regionCode',
       'delinquency_2years', 'ficoRangeLow', 'ficoRangeHigh', 'openAcc',
       'pubRec', 'revolBal', 'totalAcc', 'initialListStatus',
       'applicationType', 'title', 'issueDate_month', 'issueDate_year',
       'earliesCreditLine_month', 'earliesCreditLine_year',
       'Credit_range_months'],
      dtype='object')
The features have null data: (20,) 
 Index(['employmentLength', 'dti', 'pubRecBankruptcies', 'revolUtil', 'n0',
       'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11',
       'n12', 'n13', 'n14', 'isDefault'],
      dtype='object')


In [12]:
Y_FeatureFillClassstr = 'n0/n1/n2/n3/n4/n5/n6/n7/n8/n9/n10/n11/n12/n13/n14/employmentLength/pubRecBankruptcies'
Y_FeatureFillClass=Y_FeatureFillClassstr.split('/')
X_FeatureFill = [c for c in Full_feature if c not in (Y_FeatureFillClass+['id'])]
print('----------------------------------------------------------------------------')
print('Feature Filling Target:',len(Y_FeatureFillClass),'\n',Y_FeatureFillClass)
print('----------------------------------------------------------------------------')
print('Feature Used for filling:',len(X_FeatureFill),'\n',X_FeatureFill)
print('----------------------------------------------------------------------------')

----------------------------------------------------------------------------
Feature Filling Target: 17 
 ['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'employmentLength', 'pubRecBankruptcies']
----------------------------------------------------------------------------
Feature Used for filling: 28 
 ['loanAmnt', 'term', 'interestRate', 'installment', 'grade', 'subGrade', 'employmentTitle', 'homeOwnership', 'annualIncome', 'verificationStatus', 'purpose', 'postCode', 'regionCode', 'delinquency_2years', 'ficoRangeLow', 'ficoRangeHigh', 'openAcc', 'pubRec', 'revolBal', 'totalAcc', 'initialListStatus', 'applicationType', 'title', 'issueDate_month', 'issueDate_year', 'earliesCreditLine_month', 'earliesCreditLine_year', 'Credit_range_months']
----------------------------------------------------------------------------


In [13]:
All_data_Full = All_data[X_FeatureFill + Y_FeatureFillClass]

print('Data size before divide the Nan Data:',All_data_Full.shape)
print('----------------------------------------------------------------------------')

All_data_Null = All_data_Full[All_data_Full.isnull().T.any()]
All_data_Full = All_data_Full.dropna(axis = 0)

print('Data size after divide the Nan Data:',All_data_Full.shape)
print('The rows which have Nan Data:',All_data_Null.shape)

Data size before divide the Nan Data: (1000000, 45)
----------------------------------------------------------------------------
Data size after divide the Nan Data: (858145, 45)
The rows which have Nan Data: (141855, 45)


In [14]:
X_FillTarget=All_data_Full[X_FeatureFill]
y_FillTarget=All_data_Full[Y_FeatureFillClass]
print('Full Feature rows shape:',X_FillTarget.shape,y_FillTarget.shape)
X_Train, X_Test, y_Train, y_Test = train_test_split(X_FillTarget,y_FillTarget,
                                                    test_size = 0.2,
                                                    random_state = 233)
print('Number of Train_dataset',X_Train.shape,y_Train.shape)
print('Number of Test_dataset',X_Test.shape,y_Test.shape)
print('Random Forest Filling processing start...')

Full Feature rows shape: (858145, 28) (858145, 17)
Number of Train_dataset (686516, 28) (686516, 17)
Number of Test_dataset (171629, 28) (171629, 17)
Random Forest Filling processing start...


In [15]:
for var in Y_FeatureFillClass:
    print('The Target Feature now is:',var)
#------------------------Key code for Filling Model---------------------------#       
    Lgb = LGBMClassifier(n_estimators=15,random_state =9,n_jobs=-1)
    %time Lgb.fit(X_Train,y_Train[var])
    print("Rf filling acc score: %.3f  " % accuracy_score(y_Train[var],Lgb.predict(X_Train)),end="")
    print("Rf filling acc score: %.3f" % accuracy_score(y_Test[var],Lgb.predict(X_Test)))
    print('----------------------------------------------------------------------------')
    Fill_Temp_values = Lgb.predict(All_data_Null[All_data_Null[var].isnull().values==True][X_FeatureFill])
#------------------------Key code for Filling Model---------------------------#     
    Fill_Temp_indexs = All_data_Null[All_data_Null[var].isnull().values==True].index
    All_data_Null[var][Fill_Temp_indexs] = Fill_Temp_values 
    del Lgb,Fill_Temp_values,Fill_Temp_indexs
    
Fill_data = pd.concat([All_data_Null,All_data_Full],axis=0)
All_data[Y_FeatureFillClass] = Fill_data[Y_FeatureFillClass]
del Fill_data

The Target Feature now is: n0
Wall time: 11.8 s
Rf filling acc score: 0.699  Rf filling acc score: 0.695
----------------------------------------------------------------------------
The Target Feature now is: n1
Wall time: 3.13 s
Rf filling acc score: 0.609  Rf filling acc score: 0.606
----------------------------------------------------------------------------
The Target Feature now is: n2
Wall time: 3.22 s
Rf filling acc score: 0.705  Rf filling acc score: 0.701
----------------------------------------------------------------------------
The Target Feature now is: n3
Wall time: 3.44 s
Rf filling acc score: 0.649  Rf filling acc score: 0.647
----------------------------------------------------------------------------
The Target Feature now is: n4
Wall time: 3.39 s
Rf filling acc score: 0.702  Rf filling acc score: 0.702
----------------------------------------------------------------------------
The Target Feature now is: n5
Wall time: 3.98 s
Rf filling acc score: 0.588  Rf filling ac

### 2.6用随机森林填充缺失值(连续值)

In [16]:
Y_FeatureFillClassstr = 'dti/revolUtil'
Y_FeatureFillClass=Y_FeatureFillClassstr.split('/')
Full_feature =  All_data.describe().T['count'][All_data.describe().T['count'].values == All_data.shape[0]].index
X_FeatureFill = [c for c in Full_feature if c not in (Y_FeatureFillClass+['id'])]
print('----------------------------------------------------------------------------')
print('Feature Filling Target:',len(Y_FeatureFillClass),'\n',Y_FeatureFillClass)
print('----------------------------------------------------------------------------')
print('Feature Used for filling:',len(X_FeatureFill),'\n',X_FeatureFill)
print('----------------------------------------------------------------------------')
All_data_Full = All_data[X_FeatureFill + Y_FeatureFillClass]

print('Data size before divide the Nan Data:',All_data_Full.shape)
print('----------------------------------------------------------------------------')

All_data_Null = All_data_Full[All_data_Full.isnull().T.any()]
All_data_Full = All_data_Full.dropna(axis = 0)

print('Data size after divide the Nan Data:',All_data_Full.shape)
print('The rows which have Nan Data:',All_data_Null.shape)
X_FillTarget=All_data_Full[X_FeatureFill]
y_FillTarget=All_data_Full[Y_FeatureFillClass]
print('Full Feature rows shape:',X_FillTarget.shape,y_FillTarget.shape)
X_Train, X_Test, y_Train, y_Test = train_test_split(X_FillTarget,y_FillTarget,
                                                    test_size = 0.3,
                                                    random_state = 233)
print('Number of Train_dataset',X_Train.shape,y_Train.shape)
print('Number of Test_dataset',X_Test.shape,y_Test.shape)
print('Random Forest Filling processing start...')

----------------------------------------------------------------------------
Feature Filling Target: 2 
 ['dti', 'revolUtil']
----------------------------------------------------------------------------
Feature Used for filling: 45 
 ['loanAmnt', 'term', 'interestRate', 'installment', 'grade', 'subGrade', 'employmentTitle', 'employmentLength', 'homeOwnership', 'annualIncome', 'verificationStatus', 'purpose', 'postCode', 'regionCode', 'delinquency_2years', 'ficoRangeLow', 'ficoRangeHigh', 'openAcc', 'pubRec', 'pubRecBankruptcies', 'revolBal', 'totalAcc', 'initialListStatus', 'applicationType', 'title', 'n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'issueDate_month', 'issueDate_year', 'earliesCreditLine_month', 'earliesCreditLine_year', 'Credit_range_months']
----------------------------------------------------------------------------
Data size before divide the Nan Data: (1000000, 47)
------------------------------------------------------

In [61]:
for var in Y_FeatureFillClass:
    print('The Target Feature now is:',var)
#------------------------Key code for Filling Model---------------------------#       
    %time Lgb = RandomForestRegressor(n_estimators=15,random_state =9,n_jobs=-1).fit(X_Train,y_Train[var])
    print("Lgb acc score: %.3f" % accuracy_score(y_Test[var],Lgb.predict(X_Test)))
    print('----------------------------------------------------------------------------')
    Fill_Temp_values = Lgb.predict(All_data_Null[All_data_Null[var].isnull().values==True][X_FeatureFill])
#------------------------Key code for Filling Model---------------------------#     
    Fill_Temp_indexs = All_data_Null[All_data_Null[var].isnull().values==True].index
    All_data_Null[var][Fill_Temp_indexs] = Fill_Temp_values 
    del Lgb,Fill_Temp_values,Fill_Temp_indexs
    
Fill_data = pd.concat([All_data_Null,All_data_Full],axis=0)
All_data[Y_FeatureFillClass] = Fill_data[Y_FeatureFillClass]
del Fill_data

The Target Feature now is: employmentTitle
Wall time: 12.2 s
Lgb acc score: 8532053.870
----------------------------------------------------------------------------
The Target Feature now is: postCode
Wall time: 11.7 s
Lgb acc score: 244.680
----------------------------------------------------------------------------


In [18]:
All_data.to_csv(datapath+'All_data_NewStep3.csv',index=False)

## 3. Feature Engineering 特征工程
### 3.1第一轮特征工程（对数据进行分桶）

In [69]:
All_data = pd.read_csv(datapath + 'All_data_NewStep3.csv',encoding='utf-8',delimiter=',')
All_Y = pd.read_csv(datapath + 'Y_Final.csv',encoding='utf-8',delimiter=',')
All_data['isDefault'] = All_Y
Normal_Columns=set(All_data.columns)-set(['id','isDefault'])

In [71]:
All_data = reduce_mem_usage(All_data)
fea_importance = fast_test_lgbauc(All_data)

Memory usage of dataframe is 373.84 MB
Memory usage after optimization is: 89.65 MB
Decreased by 76.0%
All_train size (800000, 47) All_test size (200000, 47)
LGB AUC Train score: 0.7389  LGB AUC Test score: 0.7293


In [72]:
All_data['employmentTitle_count'] = All_data.groupby(['employmentTitle'])['id'].transform('count')
All_data['postCode_count'] = All_data.groupby(['postCode'])['id'].transform('count')

In [73]:
All_data['employmentTitle'][All_data['employmentTitle_count']<100] = np.nan
All_data['postCode'][All_data['postCode_count']<100] = np.nan

In [74]:
Y_FeatureFillClassstr = 'employmentTitle/postCode'
Y_FeatureFillClass=Y_FeatureFillClassstr.split('/')
Full_feature =  All_data.describe().T['count'][All_data.describe().T['count'].values == All_data.shape[0]].index
X_FeatureFill = [c for c in Full_feature if c not in (Y_FeatureFillClass+['id'])]
print('----------------------------------------------------------------------------')
print('Feature Filling Target:',len(Y_FeatureFillClass),'\n',Y_FeatureFillClass)
print('----------------------------------------------------------------------------')
print('Feature Used for filling:',len(X_FeatureFill),'\n',X_FeatureFill)
print('----------------------------------------------------------------------------')
All_data_Full = All_data[X_FeatureFill + Y_FeatureFillClass]

print('Data size before divide the Nan Data:',All_data_Full.shape)
print('----------------------------------------------------------------------------')

All_data_Null = All_data_Full[All_data_Full.isnull().T.any()]
All_data_Full = All_data_Full.dropna(axis = 0)

print('Data size after divide the Nan Data:',All_data_Full.shape)
print('The rows which have Nan Data:',All_data_Null.shape)
X_FillTarget=All_data_Full[X_FeatureFill]
y_FillTarget=All_data_Full[Y_FeatureFillClass]
print('Full Feature rows shape:',X_FillTarget.shape,y_FillTarget.shape)
X_Train, X_Test, y_Train, y_Test = train_test_split(X_FillTarget,y_FillTarget,
                                                    test_size = 0.3,
                                                    random_state = 233)
print('Number of Train_dataset',X_Train.shape,y_Train.shape)
print('Number of Test_dataset',X_Test.shape,y_Test.shape)
print('Random Forest Filling processing start...')

----------------------------------------------------------------------------
Feature Filling Target: 2 
 ['employmentTitle', 'postCode']
----------------------------------------------------------------------------
Feature Used for filling: 47 
 ['loanAmnt', 'term', 'interestRate', 'installment', 'grade', 'subGrade', 'employmentLength', 'homeOwnership', 'annualIncome', 'verificationStatus', 'purpose', 'regionCode', 'dti', 'delinquency_2years', 'ficoRangeLow', 'ficoRangeHigh', 'openAcc', 'pubRec', 'pubRecBankruptcies', 'revolBal', 'revolUtil', 'totalAcc', 'initialListStatus', 'applicationType', 'title', 'n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'issueDate_month', 'issueDate_year', 'earliesCreditLine_month', 'earliesCreditLine_year', 'Credit_range_months', 'employmentTitle_count', 'postCode_count']
----------------------------------------------------------------------------
Data size before divide the Nan Data: (1000000, 49)
-----------

In [75]:
for var in Y_FeatureFillClass:
    print('The Target Feature now is:',var)
#------------------------Key code for Filling Model---------------------------#       
    %time Lgb = LGBMClassifier(n_estimators=15,random_state =9,n_jobs=-1).fit(X_Train,y_Train[var])
    print("Lgb acc score: %.3f" % accuracy_score(y_Test[var],Lgb.predict(X_Test)))
    print('----------------------------------------------------------------------------')
    Fill_Temp_values = Lgb.predict(All_data_Null[All_data_Null[var].isnull().values==True][X_FeatureFill])
#------------------------Key code for Filling Model---------------------------#     
    Fill_Temp_indexs = All_data_Null[All_data_Null[var].isnull().values==True].index
    All_data_Null[var][Fill_Temp_indexs] = Fill_Temp_values 
    del Lgb,Fill_Temp_values,Fill_Temp_indexs
    
Fill_data = pd.concat([All_data_Null,All_data_Full],axis=0)
All_data[Y_FeatureFillClass] = Fill_data[Y_FeatureFillClass]
del Fill_data

The Target Feature now is: employmentTitle
Wall time: 2min 7s
Lgb acc score: 0.039
----------------------------------------------------------------------------
The Target Feature now is: postCode
Wall time: 1min 21s
Lgb acc score: 0.011
----------------------------------------------------------------------------


In [76]:
All_data['loanAmnt_bin2'] = np.floor_divide(All_data['loanAmnt'], 1000)
All_data['interestRate_bin2'] = np.floor_divide(All_data['interestRate'],1)
All_data['openAcc_bin2'] = np.floor_divide(All_data['openAcc'], 5)
All_data['installment_bin2'] = np.floor_divide(All_data['installment'],50)
All_data['annualIncome_bin3'] = np.floor(np.log1p(All_data['annualIncome']))
All_data['regionCode_count'] = All_data.groupby(['regionCode'])['id'].transform('count')
All_data['dti_bin3'] = np.floor(np.log1p(All_data['dti']))
All_data['revolBal_bin3'] = np.floor(np.log1p(All_data['revolBal']))
All_data['revolUtil_bin3'] = np.floor(np.log1p(All_data['revolUtil']))
All_data['totalAcc_bin2'] = np.floor_divide(All_data['totalAcc'],5)
All_data['title_count'] = All_data.groupby(['title'])['id'].transform('count')
All_data['Credit_year'] = np.floor_divide(All_data['Credit_range_months'], 12)

In [77]:
All_data['loanAmnt_bin2'] = All_data['loanAmnt_bin2'].replace({0:1})
All_data['term'] = All_data['term'].replace({3:0,5:1})
All_data['interestRate_bin2'] = np.floor_divide(All_data['interestRate'],1)
All_data['installment_bin2'].loc[All_data['installment_bin2']>25] = 25
All_data['annualIncome_bin3'].loc[All_data['annualIncome_bin3']<8] = 8
All_data['annualIncome_bin3'].loc[All_data['annualIncome_bin3']>14] = 14
All_data['dti_bin3'] = All_data['dti_bin3'].replace({-inf:0,6:5})
All_data['homeOwnership'] = All_data['homeOwnership'].replace({3:2,4:0,5:1})
All_data['purpose'] = All_data['purpose'].replace({13:3})
All_data['regionCode'] = All_data['regionCode'].replace({50:15})
All_data['delinquency_2years'].loc[All_data['delinquency_2years']>10] = 10
All_data['ficoRangeLow'].loc[All_data['ficoRangeLow']==630] = 660
All_data['ficoRangeHigh'].loc[All_data['ficoRangeHigh']==634] = 664
All_data['openAcc_bin2'].loc[All_data['openAcc_bin2']>10] = 10
All_data['pubRec'].loc[All_data['pubRec']>10] = 10
All_data['pubRecBankruptcies'].loc[All_data['pubRecBankruptcies']>5] = 5
All_data['revolBal_bin3'] = All_data['revolBal_bin3'].replace({14:13})
All_data['revolUtil_bin3'] = All_data['revolUtil_bin3'].replace({5:4,6:4})
All_data['totalAcc_bin2'].loc[All_data['totalAcc_bin2']>20] = 20
All_data['Credit_year'].loc[All_data['Credit_year']>50] = 50

In [78]:
All_data = reduce_mem_usage(All_data)
fea_importance = fast_test_lgbauc(All_data)

Memory usage of dataframe is 171.66 MB
Memory usage after optimization is: 122.07 MB
Decreased by 28.9%
All_train size (800000, 61) All_test size (200000, 61)
LGB AUC Train score: 0.7396  LGB AUC Test score: 0.7299


In [79]:
All_data.to_csv(datapath+'All_data_NewStep4.csv',index=False)

### 3.2 添加woe

In [118]:
All_data = pd.read_csv(datapath + 'All_data_NewStep4.csv',encoding='utf-8',delimiter=',')
All_Y = pd.read_csv(datapath + 'Y_Final.csv',encoding='utf-8',delimiter=',')
All_data['isDefault'] = All_Y
Data_columns_bef = list(All_data.columns)

In [119]:
def binning_cate(df,col,target='isDefault'):   
    
    total = df[target].count()
    bad = df[target].sum()
    good = total-bad
    
    group = df.groupby([col],as_index=True)
    bin_df = pd.DataFrame()

    bin_df['total'] = group[target].count()
    bin_df['totalrate'] = bin_df['total']/total
    bin_df['bad'] = group[target].sum()
    bin_df['badrate'] = bin_df['bad']/bin_df['total']
    bin_df['good'] = bin_df['total'] - bin_df['bad']
    bin_df['goodrate'] = bin_df['good']/bin_df['total']
    bin_df['badattr'] = bin_df['bad']/bad
    bin_df['goodattr'] = (bin_df['total']-bin_df['bad'])/good
    bin_df['woe'] = np.log(bin_df['badattr']/bin_df['goodattr'])
    bin_df['bin_iv'] = (bin_df['badattr']-bin_df['goodattr'])*bin_df['woe']
    bin_df['iv'] = bin_df['bin_iv'].sum()
    
    return bin_df

In [120]:
Train_df =  All_data[:len(Train_data)]
cate_col_str1 = 'loanAmnt_bin2/term/interestRate_bin2/installment_bin2/annualIncome_bin3/dti_bin3/homeOwnership'
cate_col_str2 = 'purpose/regionCode/delinquency_2years/ficoRangeLow/ficoRangeHigh/openAcc_bin2/pubRec/pubRecBankruptcies'
cate_col_str3 = 'revolBal_bin3/revolUtil_bin3/n1/n2/n3/n4/n5/n6/n7/n8/n9/n10/n11/n12/n13/n14'
cate_col_str4 = 'issueDate_month/issueDate_year/Credit_year/regionCode_count/employmentTitle_count/title_count/installment_bin2'
cate_col = cate_col_str1.split('/')+cate_col_str2.split('/')+cate_col_str3.split('/')+cate_col_str4.split('/')

cate_bin_df_list=[]
for col in cate_col:
    bin_df = binning_cate(Train_df,col,'isDefault')
    cate_bin_df_list.append(bin_df)
    
cate_iv_df = pd.DataFrame({'col':cate_col,'iv':[x['iv'].iloc[0] for x in cate_bin_df_list]}).sort_values('iv',ascending=False).reset_index(drop=True)
print('Number of feature in cate',len(cate_iv_df))
iv_select_cate_col = list(cate_iv_df[cate_iv_df.iv>0.03]['col'])
print('Select of feature in cate',len(iv_select_cate_col))

for col in iv_select_cate_col:
    woe_dict = dict([x for x in cate_bin_df_list if x.index.name==col][0]['woe'])
    All_data[col+'_woe'] = All_data[col].map(woe_dict)

Number of feature in cate 38
Select of feature in cate 14


In [121]:
All_data = reduce_mem_usage(All_data)
Data_columns_aft = list(All_data.columns)
fea_importance = fast_test_lgbauc(All_data)
Rubbish_feature = list(fea_importance.loc[fea_importance['importance']==0]['feature_name'])
New_fea = list(set(Data_columns_aft) - set(Data_columns_bef))
New_fea_drop = [i for i in New_fea if i in Rubbish_feature]
All_data = All_data.drop(New_fea_drop,axis=1)
print('Already drop the feature with no improvemnt,New Feature size:',All_data.shape[1]-2)

Memory usage of dataframe is 579.83 MB
Memory usage after optimization is: 146.87 MB
Decreased by 74.7%
All_train size (800000, 74) All_test size (200000, 74)
LGB AUC Train score: 0.7418  LGB AUC Test score: 0.7314
Already drop the feature with no improvemnt,New Feature size: 69


In [130]:
All_data['ficoRangeLow_woe'] = All_data['ficoRangeLow_woe'].fillna(0)

In [131]:
All_data.to_csv(datapath+'All_data_NewStep5.csv',index=False)

### 3.3 添加地区（regionCode）交叉函数

In [132]:
All_data = pd.read_csv(datapath + 'All_data_NewStep5.csv',encoding='utf-8',delimiter=',')
All_Y = pd.read_csv(datapath + 'Y_Final.csv',encoding='utf-8',delimiter=',')
All_data['isDefault'] = All_Y
Data_columns_bef = list(All_data.columns)

In [133]:
def feature_cross(All_data,GroupFeature='regionCode',CorssFeature='loanAmnt'):
    Train_df =  All_data[:len(Train_data)]
    Train_gb = Train_df.groupby(GroupFeature)
    all_info = {}
    for kind, kind_data in Train_gb:
    
        info = {}
        kind_data = kind_data[kind_data[CorssFeature] > 0]
        info[GroupFeature+'_'+CorssFeature+'_max'] = kind_data[CorssFeature].max()
        info[GroupFeature+'_'+CorssFeature+'_median'] = kind_data[CorssFeature].median()
        info[GroupFeature+'_'+CorssFeature+'_min'] = kind_data[CorssFeature].min()
        info[GroupFeature+'_'+CorssFeature+'_sum'] = kind_data[CorssFeature].sum()
        info[GroupFeature+'_'+CorssFeature+'_std'] = kind_data[CorssFeature].std()
        info[GroupFeature+'_'+CorssFeature+'_mean'] = kind_data[CorssFeature].mean()
        info[GroupFeature+'_'+CorssFeature+'_skew'] = kind_data[CorssFeature].skew()
        info[GroupFeature+'_'+CorssFeature+'_kurt'] = kind_data[CorssFeature].kurt()
        info[GroupFeature+'_'+CorssFeature+'_mad'] = kind_data[CorssFeature].mad()

        all_info[kind] = info
        
    brand_fe = pd.DataFrame(all_info).T.reset_index().rename(columns={"index": GroupFeature})
    All_data = All_data.merge(brand_fe, how='left', on=GroupFeature)
    
    return All_data

In [134]:
%time All_data = feature_cross(All_data,GroupFeature='regionCode',CorssFeature='loanAmnt')
%time All_data = feature_cross(All_data,GroupFeature='regionCode',CorssFeature='annualIncome')
%time All_data = feature_cross(All_data,GroupFeature='regionCode',CorssFeature='dti')
%time All_data = feature_cross(All_data,GroupFeature='regionCode',CorssFeature='revolBal')

Wall time: 5.5 s
Wall time: 2.1 s
Wall time: 2.27 s
Wall time: 2.5 s


In [135]:
All_data = reduce_mem_usage(All_data)
Data_columns_aft = list(All_data.columns)
fea_importance = fast_test_lgbauc(All_data)
Rubbish_feature = list(fea_importance.loc[fea_importance['importance']==0]['feature_name'])
New_fea = list(set(Data_columns_aft) - set(Data_columns_bef))
New_fea_drop = [i for i in New_fea if i in Rubbish_feature]
All_data = All_data.drop(New_fea_drop,axis=1)
print('Already drop the feature with no improvemnt,New Feature size:',All_data.shape[1]-2)

Memory usage of dataframe is 823.97 MB
Memory usage after optimization is: 230.79 MB
Decreased by 72.0%
All_train size (800000, 105) All_test size (200000, 105)
LGB AUC Train score: 0.7424  LGB AUC Test score: 0.7323
Already drop the feature with no improvemnt,New Feature size: 104


In [136]:
All_data.to_csv(datapath+'All_data_NewStep6.csv',index=False)

### 3.4 就业职称（employmentTitle）交叉函数

In [137]:
All_data = pd.read_csv(datapath + 'All_data_NewStep6.csv',encoding='utf-8',delimiter=',')
All_Y = pd.read_csv(datapath + 'Y_Final.csv',encoding='utf-8',delimiter=',')
All_data['isDefault'] = All_Y
Data_columns_bef = list(All_data.columns)

In [138]:
%time All_data = feature_cross(All_data,GroupFeature='employmentTitle',CorssFeature='loanAmnt')
%time All_data = feature_cross(All_data,GroupFeature='employmentTitle',CorssFeature='annualIncome')
%time All_data = feature_cross(All_data,GroupFeature='employmentTitle',CorssFeature='dti')
%time All_data = feature_cross(All_data,GroupFeature='employmentTitle',CorssFeature='revolBal')

Wall time: 9.17 s
Wall time: 4.19 s
Wall time: 4.44 s
Wall time: 4.54 s


In [139]:
All_data = reduce_mem_usage(All_data)
Data_columns_aft = list(All_data.columns)
fea_importance = fast_test_lgbauc(All_data)
Rubbish_feature = list(fea_importance.loc[fea_importance['importance']==0]['feature_name'])
New_fea = list(set(Data_columns_aft) - set(Data_columns_bef))
New_fea_drop = [i for i in New_fea if i in Rubbish_feature]
All_data = All_data.drop(New_fea_drop,axis=1)
print('Already drop the feature with no improvemnt,New Feature size:',All_data.shape[1]-2)

Memory usage of dataframe is 1091.00 MB
Memory usage after optimization is: 322.34 MB
Decreased by 70.5%
All_train size (800000, 140) All_test size (200000, 140)
LGB AUC Train score: 0.7426  LGB AUC Test score: 0.7322
Already drop the feature with no improvemnt,New Feature size: 137


In [140]:
All_data.to_csv(datapath+'All_data_NewStep7.csv',index=False)

### 3.5 邮政编码的前3位数字（postCode）交叉函数

In [141]:
All_data = pd.read_csv(datapath + 'All_data_NewStep7.csv',encoding='utf-8',delimiter=',')
All_Y = pd.read_csv(datapath + 'Y_Final.csv',encoding='utf-8',delimiter=',')
All_data['isDefault'] = All_Y
Data_columns_bef = list(All_data.columns)

In [142]:
%time All_data = feature_cross(All_data,GroupFeature='postCode',CorssFeature='loanAmnt')
%time All_data = feature_cross(All_data,GroupFeature='postCode',CorssFeature='annualIncome')
%time All_data = feature_cross(All_data,GroupFeature='postCode',CorssFeature='dti')
%time All_data = feature_cross(All_data,GroupFeature='postCode',CorssFeature='revolBal')

Wall time: 12 s
Wall time: 5.38 s
Wall time: 5.51 s
Wall time: 5.67 s


In [143]:
All_data = reduce_mem_usage(All_data)
Data_columns_aft = list(All_data.columns)
fea_importance = fast_test_lgbauc(All_data)
Rubbish_feature = list(fea_importance.loc[fea_importance['importance']==0]['feature_name'])
New_fea = list(set(Data_columns_aft) - set(Data_columns_bef))
New_fea_drop = [i for i in New_fea if i in Rubbish_feature]
All_data = All_data.drop(New_fea_drop,axis=1)
print('Already drop the feature with no improvemnt,New Feature size:',All_data.shape[1]-2)

Memory usage of dataframe is 1342.77 MB
Memory usage after optimization is: 406.27 MB
Decreased by 69.7%
All_train size (800000, 173) All_test size (200000, 173)
LGB AUC Train score: 0.7430  LGB AUC Test score: 0.7320
Already drop the feature with no improvemnt,New Feature size: 172


In [144]:
All_data.to_csv(datapath+'All_data_NewStep8.csv',index=False)

### 3.6 其他交叉函数（及补充）
homeOwnership-交叉无效<br>
verificationStatus-负向优化<br>
employmentLength-负向优化<br>
purpose-负向优化<br>
ficoRangeLow-负向优化<br>
openAcc-负向优化<br>
pubRec-负向优化<br>

In [145]:
All_data = pd.read_csv(datapath + 'All_data_NewStep8.csv',encoding='utf-8',delimiter=',')
All_Y = pd.read_csv(datapath + 'Y_Final.csv',encoding='utf-8',delimiter=',')
All_data['isDefault'] = All_Y
Data_columns_bef = list(All_data.columns)

In [146]:
%time All_data = feature_cross(All_data,GroupFeature='regionCode',CorssFeature='interestRate')
%time All_data = feature_cross(All_data,GroupFeature='employmentTitle',CorssFeature='interestRate')
%time All_data = feature_cross(All_data,GroupFeature='postCode',CorssFeature='interestRate')

Wall time: 12.3 s
Wall time: 5.48 s
Wall time: 6.42 s


In [147]:
All_data = reduce_mem_usage(All_data)
Data_columns_aft = list(All_data.columns)
fea_importance = fast_test_lgbauc(All_data)
Rubbish_feature = list(fea_importance.loc[fea_importance['importance']==0]['feature_name'])
New_fea = list(set(Data_columns_aft) - set(Data_columns_bef))
New_fea_drop = [i for i in New_fea if i in Rubbish_feature]
All_data = All_data.drop(New_fea_drop,axis=1)
print('Already drop the feature with no improvemnt,New Feature size:',All_data.shape[1]-2)

Memory usage of dataframe is 1541.14 MB
Memory usage after optimization is: 461.58 MB
Decreased by 70.0%
All_train size (800000, 199) All_test size (200000, 199)
LGB AUC Train score: 0.7432  LGB AUC Test score: 0.7321
Already drop the feature with no improvemnt,New Feature size: 192


In [148]:
All_data.to_csv(datapath+'All_data_NewStep9.csv',index=False)

### 3.7 少类型种类进行One_hot编码(反向优化最为致命)

~~~
All_data = pd.read_csv(datapath + 'All_data_NewStep9.csv',encoding='utf-8',delimiter=',')
All_Y = pd.read_csv(datapath + 'Y_Final.csv',encoding='utf-8',delimiter=',')
All_data['isDefault'] = All_Y
Data_columns_bef = list(All_data.columns)
~~~

~~~
All_data['ficoRangeMinus']=All_data['ficoRangeHigh']-All_data['ficoRangeLow']

onehot_columnstr='term/grade/homeOwnership/verificationStatus/ficoRangeMinus/initialListStatus/applicationType'
onehot_column = onehot_columnstr.split('/')
All_data = pd.get_dummies(All_data,columns=onehot_column,drop_first=True)
~~~

~~~
All_data = reduce_mem_usage(All_data)
Data_columns_aft = list(All_data.columns)
fea_importance = fast_test_lgbauc(All_data)
Rubbish_feature = list(fea_importance.loc[fea_importance['importance']==0]['feature_name'])
New_fea = list(set(Data_columns_aft) - set(Data_columns_bef))
New_fea_drop = [i for i in New_fea if i in Rubbish_feature]
All_data = All_data.drop(New_fea_drop,axis=1)
print('Already drop the feature with no improvemnt,New Feature size:',All_data.shape[1]-2)

~~~

### 3.8 特征选择(删除重要性小于5的垃圾特征)

In [149]:
All_data = pd.read_csv(datapath + 'All_data_NewStep9.csv',encoding='utf-8',delimiter=',')
All_Y = pd.read_csv(datapath + 'Y_Final.csv',encoding='utf-8',delimiter=',')
#All_data = All_data.drop(['id','isDefault'],axis=1)
All_data = reduce_mem_usage(All_data)

Memory usage of dataframe is 1480.10 MB
Memory usage after optimization is: 436.78 MB
Decreased by 70.5%


In [150]:
Data_columns_all = list(All_data.columns)
fea_importance = fast_test_lgbauc(All_data)
Rubbish_feature = list(fea_importance.loc[fea_importance['importance']<5]['feature_name'])
New_fea = list(set(Data_columns_all) - set(Rubbish_feature))
All_data = All_data.drop(Rubbish_feature,axis=1)
print('Already drop the feature with no improvemnt,New Feature size:',All_data.shape[1]-2)
fea_importance = fast_test_lgbauc(All_data)

All_train size (800000, 192) All_test size (200000, 192)
LGB AUC Train score: 0.7432  LGB AUC Test score: 0.7321
Already drop the feature with no improvemnt,New Feature size: 104
All_train size (800000, 104) All_test size (200000, 104)
LGB AUC Train score: 0.7431  LGB AUC Test score: 0.7325


In [151]:
All_data.to_csv(datapath+'All_data_NewStep10.csv',index=False)

## 4.模型训练及参数查找(lgb)

### 4.1 LGB最佳参数测试

In [156]:
All_data = pd.read_csv(datapath + 'All_data_NewStep10.csv',encoding='utf-8',delimiter=',')
All_Y = pd.read_csv(datapath + 'Y_Final.csv',encoding='utf-8',delimiter=',')
All_data = All_data.drop(['id','isDefault'],axis=1)
All_data = reduce_mem_usage(All_data)

n_train =len(All_Y)

All_train =  All_data[:n_train] 
All_test = All_data[n_train:] 
print('All_train size',All_train.shape,'All_test size',All_test.shape)
del All_data

X_Train = All_train
y_Train = All_Y
X_Test = All_test
print('--------------------------TrainDataset--------------------------')
print('xtrain size is : {}'.format(X_Train.shape),'ytrain size is : {}'.format(y_Train.shape))
print('--------------------------TrainDataset--------------------------')
print('xtest size is : {}'.format(X_Test.shape))

Memory usage of dataframe is 793.46 MB
Memory usage after optimization is: 217.44 MB
Decreased by 72.6%
All_train size (800000, 104) All_test size (200000, 104)
--------------------------TrainDataset--------------------------
xtrain size is : (800000, 104) ytrain size is : (800000, 1)
--------------------------TrainDataset--------------------------
xtest size is : (200000, 104)


In [ ]:
import lightgbm as lgb
params_dic = {    
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'auc',
          'nthread':4,
          'learning_rate':0.1,
          'num_leaves':30, 
          'max_depth': 5,   
          'subsample': 0.8, 
          'colsample_bytree': 0.8, 
    }

data_train = lgb.Dataset(X_Train, y_Train)
cv_results = lgb.cv(params=params_dic,train_set = data_train, num_boost_round=1000, nfold=5, stratified=False, 
                    shuffle=True, metrics='auc',early_stopping_rounds=50,
                    verbose_eval = 0,seed=0)
print('best n_estimators:', len(cv_results['auc-mean']))
print('best cv score:', pd.Series(cv_results['auc-mean']).max())

In [175]:
#由于搜索太慢采用贪心策略进行搜索（下为初始参数）
initial_params ={'boosting_type':'gbdt',
                 'objective':'binary',
                 'metrics':'auc',
                 'learning_rate':0.1,
                 'n_estimators':575,
                 'verbose':-1}


# Single Wall time: 4min 27s
#下面为需要调整的参数
params_test1={'max_depth': range(3,8,1), 'num_leaves':range(5,100,5)}
params_test2={'max_bin': range(5,256,10), 'min_data_in_leaf':range(1,102,10)}

params_test3={'feature_fraction': [0.6,0.7,0.8,0.9,1.0],
              'bagging_fraction': [0.6,0.7,0.8,0.9,1.0],
              'bagging_freq': range(0,81,10)}

params_test4={'lambda_l1': [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0],
              'lambda_l2': [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0]}

params_test5={'min_split_gain':[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]}

params_turns = 4

#开始采用贪心算法对参数进行搜索，返回的initial_params为获取的最佳参数
df_cv_result=[]
for i in range(params_turns):
    params_string = "params_test"+str(i+1)
    params_round = locals()[params_string]
    print('----------------------Finding Parameters,',params_string,'--------------------------') 
    print('Parameters:',params_string,params_round)
    
    gsearch = GridSearchCV(estimator = LGBMClassifier(**initial_params), 
                       param_grid = params_round, scoring='roc_auc',cv=5,n_jobs=-1)
    
    print('Start Fiting,',params_string)          
    %time gsearch.fit(X_Train,y_Train)
    print(gsearch.best_params_,'Best evaluation Scores',gsearch.best_score_)
    initial_params.update(gsearch.best_params_)
    
    df_cv_result.append(pd.DataFrame(gsearch.cv_results_))
    
    

----------------------Finding Parameters, params_test1 --------------------------
Parameters: params_test1 {'max_depth': range(3, 8), 'num_leaves': range(5, 100, 5)}
Start Fiting, params_test1


KeyboardInterrupt: 

### 4.1 最佳LGB参数(未加n_estimators)
```
{'boosting_type': 'gbdt',
 'objective': 'binary',
 'metrics': 'auc',
 'learning_rate': 0.1,
 'n_estimators': 575,
 'max_depth': 7,
 'num_leaves': 80,
 'verbose': -1,
 'max_bin': 155,
 'min_data_in_leaf': 101,
 'bagging_fraction': 0.6,
 'bagging_freq': 0,
 'feature_fraction': 1.0,
 'lambda_l1': 0.5,
 'lambda_l2': 0.1,
 'min_split_gain': 0.0}
```

### 4.2 CatBoost调试最佳参数 (谷歌云上运行)
__CatB AUC Train score: 0.7834 CatB AUC Test score: 0.7363__

```
#由于搜索太慢采用贪心策略进行搜索（下为初始参数）

initial_params ={'eval_metric':'AUC',
                 'random_seed':42,                
                 'verbose':0}

#下面为需要调整的参数

params_test1={'n_estimators':[10,20,50,100,200,500,1000]}
params_test2={'depth': range(3,8,1)}
params_test3={'learning_rate':[0.001,0.01,0.1,0.5,0.7,0.8,0.9,1.0]}
params_test4={'l2_leaf_reg':[0.01,0.1,1,10,100]}
params_test5={'border_count':[128,254]}


params_turns = 5

#开始采用贪心算法对参数进行搜索，返回的initial_params为获取的最佳参数
df_cv_result=[]
for i in range(params_turns):
    params_string = "params_test"+str(i+1)
    params_round = locals()[params_string]
    print('----------------------Finding Parameters,',params_string,'--------------------------') 
    print('Parameters:',params_string,params_round)
    
    gsearch = GridSearchCV(estimator = CatBoostClassifier(**initial_params), 
                       param_grid = params_round, scoring='roc_auc',cv=5,n_jobs=-1)
    
    print('Start Fiting,',params_string)          
    %time gsearch.fit(X_Train,y_Train)
    print(gsearch.best_params_,'Best evaluation Scores',gsearch.best_score_)
    initial_params.update(gsearch.best_params_)
    
    df_cv_result.append(pd.DataFrame(gsearch.cv_results_))
```

此处代码为谷歌云上运行，云上结果如下
```
----------------------Finding Parameters, params_test1 --------------------------
Parameters: params_test1 {'n_estimators': [10, 20, 50, 100, 200, 500, 1000]}
Start Fiting, params_test1
CPU times: user 21min 13s, sys: 23 s, total: 21min 36s
Wall time: 13min 28s
{'n_estimators': 1000} Best evaluation Scores 0.7370597109406334
----------------------Finding Parameters, params_test2 --------------------------
Parameters: params_test2 {'depth': range(3, 8)}
Start Fiting, params_test2
CPU times: user 15min 58s, sys: 17 s, total: 16min 15s
Wall time: 26min 33s
{'depth': 4} Best evaluation Scores 0.7379032052238406
----------------------Finding Parameters, params_test3 --------------------------
Parameters: params_test3 {'learning_rate': [0.001, 0.01, 0.1, 0.5, 0.7, 0.8, 0.9, 1.0]}
Start Fiting, params_test3
CPU times: user 16min 8s, sys: 16.5 s, total: 16min 25s
Wall time: 37min 8s
{'learning_rate': 0.1} Best evaluation Scores 0.7378817116059709
----------------------Finding Parameters, params_test4 --------------------------
Parameters: params_test4 {'l2_leaf_reg': [0.01, 0.1, 1, 10, 100]}
Start Fiting, params_test4
CPU times: user 16min 34s, sys: 18.9 s, total: 16min 53s
Wall time: 23min 7s
{'l2_leaf_reg': 100} Best evaluation Scores 0.7379399636868315
----------------------Finding Parameters, params_test5 --------------------------
Parameters: params_test5 {'border_count': [128, 254]}
Start Fiting, params_test5
CPU times: user 16min 25s, sys: 17.8 s, total: 16min 43s
Wall time: 10min 17s
{'border_count': 128} Best evaluation Scores 0.7380043408466763
```

### 4.2 获取的最佳cat参数为
```
{'loss_function': 'Logloss',
 'learning_rate': 0.1,
 'random_seed': 42,
 'n_estimators': 931,
 'depth': 6,
 'verbose': 0,
 'bootstrap_type': 'MVS',
 'border_count': 145,
 'min_data_in_leaf': 1,
 'l2_leaf_reg': 100}
 ```

### 4.3最佳LogisticRegression参数查找

In [170]:
#由于搜索太慢采用贪心策略进行搜索（下为初始参数）
initial_params ={
    
}


# Single Wall time: 4min 27s
#下面为需要调整的参数
params_test1={'C': [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0,10,100,1000]}


params_turns = 1

#开始采用贪心算法对参数进行搜索，返回的initial_params为获取的最佳参数
df_cv_result=[]
for i in range(params_turns):
    params_string = "params_test"+str(i+1)
    params_round = locals()[params_string]
    print('----------------------Finding Parameters,',params_string,'--------------------------') 
    print('Parameters:',params_string,params_round)
    
    gsearch = GridSearchCV(estimator = LogisticRegression(**initial_params), 
                       param_grid = params_round, scoring='roc_auc',cv=5,n_jobs=-1)
    
    print('Start Fiting,',params_string)          
    %time gsearch.fit(X_Train,y_Train)
    print(gsearch.best_params_,'Best evaluation Scores',gsearch.best_score_)
    initial_params.update(gsearch.best_params_)
    
    df_cv_result.append(pd.DataFrame(gsearch.cv_results_))
    

----------------------Finding Parameters, params_test1 --------------------------
Parameters: params_test1 {'C': [1e-05, 0.001, 0.1, 0.0, 0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 10, 100, 1000]}
Start Fiting, params_test1
Wall time: 23min 29s
{'C': 100} Best evaluation Scores 0.6001374029563713


In [29]:
lgb_params = {
'boosting_type': 'gbdt',
'objective': 'binary',
'metrics': 'auc',
'learning_rate': 0.1,
'n_estimators': 631,
'max_depth': 5,
'num_leaves': 35,
'verbose': -1,
'max_bin': 155,
'min_data_in_leaf': 101,
'bagging_fraction': 0.6,
'bagging_freq': 0,
'feature_fraction': 1.0,
'lambda_l1': 0.5,
'lambda_l2': 0.1,
'min_split_gain': 0.0,
'n_jobs':-1
             }

CatB_params = {
'loss_function': 'Logloss',
'learning_rate': 0.1,
'n_estimators': 931,
'depth': 6,
'verbose': 0,
'bootstrap_type': 'MVS',
'border_count': 145,
'min_data_in_leaf': 1,
'l2_leaf_reg': 100
}

rf_params = {
'n_estimators':300,                   
'criterion':'gini',                   
'max_depth':15,                   
'min_samples_split':50,                   
'min_samples_leaf':20,                   
'max_features':0.5,                   
'max_leaf_nodes':None,                   
'min_impurity_decrease':0.0,                   
'n_jobs':-1}

## 5. Stacking Model
### 5.1 定义一些Stacking 模型的基础函数和类

In [16]:
# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=SEED, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def predict_proba(self, x):
        return self.clf.predict_proba(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
        
def get_oof(clf, X_Train, y_Train, X_Test):
    oof_train = np.zeros((n_train,))
    oof_test = np.zeros((n_test,))
    oof_test_skf = np.empty((NFOLDS, n_test))
    X_Train.index = range(len(X_Train))
    y_Train.index = range(len(y_Train))
    for i, (train_index, test_index) in enumerate(kf.split(X_Train)):
        X_tr = X_Train.loc[train_index]
        y_tr = y_Train.loc[train_index]
        X_te = X_Train.loc[test_index]

        clf.train(X_tr, y_tr)

        oof_train[test_index] = clf.predict_proba(X_te)[:,1]
        oof_test_skf[i, :] = clf.predict_proba(X_Test)[:,1]

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [17]:
All_data = pd.read_csv(datapath + 'All_data_NewStep10.csv',encoding='utf-8',delimiter=',')
All_Y = pd.read_csv(datapath + 'Y_Final.csv',encoding='utf-8',delimiter=',')
All_data = All_data.drop(['id','isDefault'],axis=1)
All_data = reduce_mem_usage(All_data)

n_train =len(All_Y)

All_train =  All_data[:n_train] 
All_test = All_data[n_train:] 
print('All_train size',All_train.shape,'All_test size',All_test.shape)
del All_data

X_Train = All_train
y_Train = All_Y
X_Test = All_test
print('--------------------------TrainDataset--------------------------')
print('xtrain size is : {}'.format(X_Train.shape),'ytrain size is : {}'.format(y_Train.shape))
print('--------------------------TrainDataset--------------------------')
print('xtest size is : {}'.format(X_Test.shape))

Memory usage of dataframe is 793.46 MB
Memory usage after optimization is: 217.44 MB
Decreased by 72.6%
All_train size (800000, 104) All_test size (200000, 104)
--------------------------TrainDataset--------------------------
xtrain size is : (800000, 104) ytrain size is : (800000, 1)
--------------------------TrainDataset--------------------------
xtest size is : (200000, 104)


In [18]:
n_train = X_Train.shape[0]
n_test = X_Test.shape[0]
SEED = 42
NFOLDS = 5
kf = KFold(n_splits= NFOLDS,random_state=SEED)

In [19]:
lgb = SklearnHelper(clf=LGBMClassifier,seed=SEED,params=lgb_params)
%time lgb_oof_train, lgb_oof_test = get_oof(lgb, X_Train, y_Train, X_Test)
print("lgb single AUC score: %.4f" % roc_auc_score(y_Train,lgb.predict_proba(X_Train)[:,1]))

[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_data_in_leaf is set=101, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=101
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_data_in_leaf is set=101, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=101
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored

In [23]:
CatB = SklearnHelper(clf=CatBoostClassifier,seed=SEED,params=CatB_params)
%time CatB_oof_train, CatB_oof_test = get_oof(CatB, X_Train, y_Train, X_Test)
print("CatB single AUC score: %.4f" % roc_auc_score(y_Train,CatB.predict_proba(X_Train)[:,1]))

Wall time: 10min 14s
CatB single AUC score: 0.7557


In [30]:
rf = SklearnHelper(clf=RandomForestClassifier,seed=SEED,params=rf_params)
%time rf_oof_train, rf_oof_test = get_oof(rf, X_Train, y_Train, X_Test)
print("rf single AUC score: %.4f" % roc_auc_score(y_Train,rf.predict_proba(X_Train)[:,1]))

Wall time: 1h 2min 9s
rf single AUC score: 0.8051


### 5.2归一化 MLP训练

In [41]:
All_data = pd.read_csv(datapath + 'All_data_NewStep10.csv',encoding='utf-8',delimiter=',')
All_Y = pd.read_csv(datapath + 'Y_Final.csv',encoding='utf-8',delimiter=',')
All_data = All_data.drop(['id','isDefault'],axis=1)
All_data = reduce_mem_usage(All_data)

n_train =len(All_Y)

##对数据进行归一化
minmax_scale = MinMaxScaler(feature_range=(0, 1))
All_data = minmax_scale.fit_transform(All_data)

All_train =  pd.DataFrame(All_data[:n_train]) 
All_test = pd.DataFrame(All_data[n_train:])
print('All_train size',All_train.shape,'All_test size',All_test.shape)
del All_data

Memory usage of dataframe is 793.46 MB
Memory usage after optimization is: 217.44 MB
Decreased by 72.6%
All_train size (800000, 104) All_test size (200000, 104)


In [45]:
mlp_params={
'hidden_layer_sizes':(128,64,32,16),                   
'activation':'relu',                   
'solver':'adam',
'alpha':0.0001,
'batch_size':200,
'learning_rate':'adaptive',
'learning_rate_init':0.001,
'max_iter':5,
'early_stopping':True,
'n_iter_no_change':10,
'beta_1':0.9,
'beta_2':0.999,
'verbose':1,
'epsilon':1e-8}

mlp = SklearnHelper(clf=MLPClassifier,seed=SEED,params=mlp_params)
%time mlp_oof_train, mlp_oof_test = get_oof(mlp, X_Train, y_Train, X_Test)
print("lgb single AUC score: %.4f" % roc_auc_score(y_Train,mlp.predict_proba(X_Train)[:,1]))

In [31]:
np.save(datapath +'lgb_oof_train.npy',lgb_oof_train)
np.save(datapath +'lgb_oof_test.npy',lgb_oof_test)
np.save(datapath +'CatB_oof_train.npy',CatB_oof_train)
np.save(datapath +'CatB_oof_test.npy',CatB_oof_test)
np.save(datapath +'rf_oof_train.npy',rf_oof_train)
np.save(datapath +'rf_oof_test.npy',rf_oof_test)

### 5.2 开始训练Stacking模型

In [47]:
base_predictions_train = pd.DataFrame({'LGBMClassifier':lgb_oof_train.ravel(),
                                      'CatBoostClassifier':CatB_oof_train.ravel(),
                                      'RandomForestClassifier':rf_oof_train.ravel()})
base_predictions_train.head()

,LGBMClassifier,CatBoostClassifier,RandomForestClassifier
0,0.261008,0.277994,0.295534
1,0.360507,0.244783,0.307828
2,0.400846,0.414225,0.410436
3,0.050994,0.063231,0.055297
4,0.425535,0.370200,0.339742


In [48]:
X2_Train = np.concatenate((lgb_oof_train,CatB_oof_train,rf_oof_train),axis=1)
X2_Test = np.concatenate((lgb_oof_test,CatB_oof_test,rf_oof_test),axis=1)

In [49]:
lr2 = LGBMClassifier().fit(X2_Train, y_Train)
#LogisticRegression,LGBMClassifier
print("LR stacking AUC score: %.4f" % roc_auc_score(y_Train,lr2.predict_proba(X2_Train)[:,1]))

LR stacking AUC score: 0.7415


In [51]:
Test_data =pd.read_csv('testA.csv',encoding='utf-8',delimiter=',' )
Test_data['isDefault']=lr2.predict_proba(X2_Test)[:,1]
Test_data[['id','isDefault']].to_csv('FR_waittest2.csv',index=False)